In [2]:
import numpy as np
import REMG
import time
REMG.list_devices()

,device,name,description,hwid,vid,pid,serial_number,location,manufacturer,product,interface
0,/dev/cu.BLTH,cu.BLTH,n/a,n/a,NaN,NaN,None,None,None,None,None
1,/dev/cu.Bluetooth-Incoming-Port,cu.Bluetooth-Incoming-Port,n/a,n/a,NaN,NaN,None,None,None,None,None
2,/dev/cu.BoseQC35II,cu.BoseQC35II,n/a,n/a,NaN,NaN,None,None,None,None,None
3,/dev/cu.usbserial-143100,cu.usbserial-143100,Dual RS232-HS - Dual RS232-HS,USB VID:PID=0403:6010 LOCATION=20-3.1,1027.0,24592.0,None,20-3.1,FTDI,Dual RS232-HS,Dual RS232-HS
4,/dev/cu.usbserial-143101,cu.usbserial-143101,Dual RS232-HS - Dual RS232-HS,USB VID:PID=0403:6010 LOCATION=20-3.1,1027.0,24592.0,None,20-3.1,FTDI,Dual RS232-HS,Dual RS232-HS


In [3]:
rheo = REMG.remgSerialInterface('/dev/cu.usbserial-143101')

Opened serial port /dev/cu.usbserial-143101 at 921600 baud


In [4]:
rheo.start_reading()

Started Serial While Loop
Started reading thread


In [11]:
while True:
    if len(rheo.inc_data)>0:
        pack = rheo.inc_data.popleft()
        print(pack[0], list(pack[1][rheo.EMG_IDX]), list(pack[1][rheo.RMG_IDX]), end=" "*50+'\r')
    time.sleep(0.005)


KeyboardInterrupt: 

# Teleplot


In [24]:

# '\n'.join(list(map(lambda x: f'{x[0]}:{x[1]}', zip(RMG_LABELS, pack[1][rheo.RMG_IDX]))))


'RMG0:102494273\nRMG1:113865794\nRMG2:123017027\nRMG3:113799748\nRMG4:110046277\nRMG5:93094726\nRMG6:103005767\nRMG7:100206408\nRMG8:103290697\nRMG9:106932298\nRMG10:119899211\nRMG11:76'

In [26]:
import socket
teleplotAddr = ("127.0.0.1",47269)
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

def sendTelemetry(name, value):
    now = time.time() * 1000
    msg = name+":"+str(now)+":"+str(value)+"|g"
    sock.sendto(msg.encode(), teleplotAddr)

def sendManyTelemetry(s: str):
    sock.sendto(s.encode(), teleplotAddr)
EMG_LABELS= ["EMG%i"%i for i in range(len(rheo.EMG_IDX))]
RMG_LABELS = ["RMG%i"%i for i in range(len(rheo.RMG_IDX))]

while True:
    if len(rheo.inc_data)>0:
        pack = rheo.inc_data.popleft()
        print(pack[0], list(pack[1][rheo.EMG_IDX]), list(pack[1][rheo.RMG_IDX]), end=" "*50+'\r')
        sendManyTelemetry('\n'.join(list(map(lambda x: f'{x[0]}:{x[1]}', zip(EMG_LABELS, pack[1][rheo.EMG_IDX])))))
        sendManyTelemetry('\n'.join(list(map(lambda x: f'{x[0]}:{x[1]}', zip(RMG_LABELS, pack[1][rheo.RMG_IDX])))))
    time.sleep(0.005)


KeyboardInterrupt: 